# Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import random
from datetime import timedelta, datetime
from datetime import date


# Define Constants

In [2]:
NUM_PATIENTS = 5000
MIN_AGE = 16
MAX_AGE = 65
TEMP_NORMAL_C = (36.5, 37.2)
TEMP_FEVER_C = (38, 41.0)
TEMP_HYPOTHERMIA_C = (32.0, 35.0)
BP_NORMAL_SYSTOLIC = (90, 120)
BP_NORMAL_DIASTOLIC = (60, 80)
HEIGHT_ADULT_RANGE_CM = (147, 195)
BMI_CATEGORIES = {
    "Underweight": (None, 18.5),
    "Normal": (18.5, 24.9),
    "Overweight": (25, 29.9),
    "Obese": (30, None)
}
SAMPLES_PER_PATIENT = 12
START_DATE = date(2024, 2, 15) # Instantiate with year, month, day


# Helper Functions

In [3]:
def generate_age():
    return random.randint(MIN_AGE, MAX_AGE)

def generate_temperature(age):
    status = random.choices(['normal', 'fever', 'hypothermia'], weights=[0.85, 0.1, 0.05])[0]
    if status == 'fever':
        return int(round(random.uniform(*TEMP_FEVER_C)))
    elif status == 'hypothermia':
        return int(round(random.uniform(*TEMP_HYPOTHERMIA_C)))
    else:
        return int(round(random.uniform(*TEMP_NORMAL_C)))

def generate_blood_pressure(age, week_num):
    systolic = random.randint(*BP_NORMAL_SYSTOLIC)
    diastolic = random.randint(*BP_NORMAL_DIASTOLIC)
    if age > 50:
        systolic += 10
        diastolic += 5

    if week_num > 6:   # Simulate slightly rising BP
        systolic += week_num
        diastolic += 2

    return f"{systolic}/{diastolic}"

def generate_heart_rate(age):
    base_rate = random.randint(60, 100)
    if age > 50:
        base_rate += random.randint(0, 10)
    return base_rate

def generate_respiration_rate(age):
    base_rate = random.randint(12, 20)
    if age > 60:
        base_rate += random.randint(0, 5)
    return base_rate

def generate_height_weight():
    height_cm = random.randint(*HEIGHT_ADULT_RANGE_CM)
    bmi_category = random.choices(list(BMI_CATEGORIES.keys()), weights=[0.1, 0.6, 0.2, 0.1])[0]
    bmi_range = BMI_CATEGORIES[bmi_category]
    min_bmi = 15 if bmi_range[0] is None else bmi_range[0]
    max_bmi = 40 if bmi_range[1] is None else bmi_range[1]
    bmi = random.uniform(min_bmi, max_bmi)
    weight_kg = (bmi * (height_cm / 100) ** 2)
    return height_cm, int(weight_kg)

def calculate_bmi(weight_kg, height_cm):
    return weight_kg / ((height_cm / 100) ** 2)

def determine_risk_level(bmi, blood_pressure, age):
    if bmi > 30 or blood_pressure >= "140/90" or age > 60:
        return "High Risk for CVD"
    elif bmi > 25 or blood_pressure >= "120/80" or age > 50:
        return "Moderate Risk for CVD or Diabetes"
    else:
        return "Low Risk"

def simulate_treatment_action(risk_level):
    if risk_level in ["High Risk for CVD", "Moderate Risk for CVD or Diabetes"]:
        return "Administer Medication"
    return "Monitor"

def calculate_reward(previous_risk_level, current_risk_level):
    risk_to_int = {"Low Risk": 0, "Elevated Risk": 1, "Moderate Risk for CVD or Diabetes": 2, "High Risk for CVD": 3}
    return risk_to_int[previous_risk_level] - risk_to_int[current_risk_level]

# Data Generation Function

In [4]:
data = []
for patient_id in range(1, NUM_PATIENTS + 1):
    age = generate_age()
    height_cm, weight_kg = generate_height_weight()

    base_date = START_DATE
    for i in range(SAMPLES_PER_PATIENT):
        temp_c = generate_temperature(age)
        bp = generate_blood_pressure(age, i)
        heart_rate = generate_heart_rate(age)
        respiration_rate = generate_respiration_rate(age)

        bmi = calculate_bmi(weight_kg, height_cm)
        risk_level = determine_risk_level(bmi, bp, age)
        action = simulate_treatment_action(risk_level)

        data.append({
            "Patient ID": f"PAT_{patient_id:05d}",
            "Date": base_date.strftime('%Y-%m-%d'),
            "Age": age,
            "Temperature_C": temp_c,
            "Blood Pressure": bp,
            "Heart Rate": heart_rate,
            "Respiration Rate": respiration_rate,
            "Height_cm": height_cm,
            "Weight_kg": weight_kg,
            "BMI": bmi,
            "Risk Level": risk_level,
            "Action": action
        })

        base_date += timedelta(days=7)

# Create Dataframe

In [5]:
df = pd.DataFrame(data)
df.head()

,Patient ID,Date,Age,Temperature_C,Blood Pressure,Heart Rate,Respiration Rate,Height_cm,Weight_kg,BMI,Risk Level,Action
0,PAT_00001,2024-02-15,50,35,101/67,93,20,165,51,18.732782,Low Risk,Monitor
1,PAT_00001,2024-02-22,50,37,112/79,87,20,165,51,18.732782,Low Risk,Monitor
2,PAT_00001,2024-02-29,50,37,114/61,89,12,165,51,18.732782,Low Risk,Monitor
3,PAT_00001,2024-03-07,50,37,103/64,74,17,165,51,18.732782,Low Risk,Monitor
4,PAT_00001,2024-03-14,50,37,101/65,60,19,165,51,18.732782,Low Risk,Monitor


# Export to CSV

In [7]:
df.to_csv('medical_data_with_trend.csv', index=False)